# Louvain Community Detection


In this notebook, we will use cuGraph to identify the cluster in a test graph using the Louvain algorithm  


## Introduction

The Louvain method of community detection is a greedy heirarical clsutering algorithm which seeks to optimize Modularity as it progresses. Louvain starts with each vertex in its own clusters and iteratively merges groups using graph contraction.  

For a detailed description of the algorithm see: https://en.wikipedia.org/wiki/Louvain_Modularity

It takes as input a cugraph.Graph object and returns as output a 
cudf.Dataframe object with the id and assigned partition for each 
vertex as well as the final modularity score


To compute the Louvain cluster in cuGraph use: <br>

**nvLouvain(G)**
* __G__: A cugraph.Graph object

Returns:

* tupal __lovain dataframe__ and __modularity__


* __louvain__: cudf.DataFrame with two names columns:
    * louvain["vertex"]: The vertex id.
    * louvain["partition"]: The assigned partition.
    
* __modularity__ : the overall modularity of the graph

All vertices with the same partition ID are in the same cluster


## cuGraph 0.8 Notes
cuGraph version 0.8 has some limitations:
* Only Int32 Vertex ID are supported
* Only float (FP32) edge data is supported
* Vertex numbering is assumed to start at zero

These limitations are being addressed and will be fixed future versions.  
These example notebooks will illustrate how to manipulate the data so that it comforms to the current limitations    

A new renumbering feature is being worked and will be reflected in updated notebooks for the next release.

## References

* Blondel, V. D., Guillaume, J.-L., Lambiotte, R., and Lefebvre, E. Fast unfolding of communities in large networks. Journal of statistical mechanics: theory and experiment 2008, 10 (2008), P10008.


### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](./img/zachary_black_lines.png)


### Preparation

In [1]:
# Import needed libraries
import cugraph
import cudf
import numpy as np
from collections import OrderedDict

## Read data using cuDF

Remember that the inpud cuDF dataframe must contain one column for source, and one for destination

In [2]:
# Test file    
datafile='../../../data/karate-data.csv'

# Read the data file
cols = ["src", "dst"]

dtypes = OrderedDict([
        ("src", "int32"), 
        ("dst", "int32")
        ])

gdf = cudf.read_csv(datafile, names=cols, delimiter='\t', dtype=list(dtypes.values()) )

We use the capabilities of cuDF to make the id numbers of the nodes to start from zero:

In [3]:
# Louvain is dependent on vertex ID starting at zero
gdf["src_0"] = gdf["src"] - 1
gdf["dst_0"] = gdf["dst"] - 1

Additionally cugraph's Louvain algorithm is capable to deal with generally weighted graphs, so we just need to add a weight of 1.0 to every vertex:

In [4]:
# The algorithm also requires that there are vertex weights.  Just use 1.0 
gdf["data"] = 1.0

In [5]:
# just for sanity check, let's look at the data types in the dataframe
gdf.dtypes

src        int32
dst        int32
src_0      int32
dst_0      int32
data     float64
dtype: object

Time to create our cugraph Graph:

In [6]:
# create a Graph 
G = cugraph.Graph()
G.from_cudf_edgelist(gdf, source='src_0', destination='dst_0', edge_attr='data')

Now the call to the Louvain algorithm is very simple:

In [7]:
# Call Louvain on the graph
df, mod = cugraph.louvain(G) 

The modularity is one measure of the structure of networks or graphs. It was designed to measure the strength of division of a network into communities:

In [8]:
# Print the modularity score
print('Modularity was {}'.format(mod))
print()

Modularity was 0.4027777777777778



In [9]:
df.dtypes

vertex       int32
partition    int32
dtype: object

Now lets see how many partitions or communities did the algorithm find:

In [10]:
# How many partitions where found
part_ids = df["partition"].unique()

In [11]:
print(str(len(part_ids)) + " partitions detected")

4 partitions detected


Finally lets actually see our communities!

In [12]:
for p in range(len(part_ids)):
    part = []
    for i in range(len(df)):
        if (df['partition'][i] == p):
            part.append(df['vertex'][i] +1)
    print("Partition " + str(p) + ":")
    print(part)


Partition 0:
[1, 2, 3, 4, 8, 10, 12, 13, 14, 18, 20, 22]
Partition 1:
[5, 6, 7, 11, 17]
Partition 2:
[9, 15, 16, 19, 21, 23, 27, 29, 30, 31, 32, 33, 34]
Partition 3:
[24, 25, 26, 28]


Here is how the communities look like:

![Karate Club](./img/comms.jpeg)